# Homework Submission

```ascii
    ┌─────────────┐           ┌─────────────┐
    │             │           │             │
    │  HOME       │ ────────► │  WORK       │
    │             │           │             │
    └─────────────┘           └─────────────┘
```

## Question 1. dlt Version

---


What's the version of dlt that you installed?
- `Version: 1.12.3`

In [1]:
!pip show dlt

Name: dlt
Version: 1.12.3
Summary: dlt is an open-source python-first scalable data loading library that does not require any backend to run.
Home-page: https://github.com/dlt-hub
Author: 
Author-email: "dltHub Inc." <services@dlthub.com>
License-Expression: Apache-2.0
Location: /home/radianv/anaconda3/envs/llm-zoomcamp/lib/python3.10/site-packages
Requires: click, fsspec, gitpython, giturlparse, hexbytes, humanize, jsonpath-ng, orjson, packaging, pathvalidate, pendulum, pluggy, pytz, pyyaml, requests, requirements-parser, rich-argparse, semver, setuptools, simplejson, sqlglot, tenacity, tomlkit, typing-extensions, tzdata
Required-by: cognee


## Question 2. dlt pipeline

---
How many rows were inserted into the `zoomcamp_data` collection?

Look for **"Normalized data for the following tables:"** in the trace output.

### Step 1 - support functions

In [10]:
import dlt
import requests
import pandas as pd
from datetime import datetime
from qdrant_client import QdrantClient, models
from dlt.destinations import qdrant

In [8]:
qd_client = QdrantClient("http://localhost:6333") #connecting to local Qdrant instance
EMBEDDING_DIMENSIONALITY = 512
model_handle = "BAAI/bge-small-en"

In [4]:
@dlt.resource(write_disposition="replace", name="zoomcamp_data")
def zoomcamp_data():
    docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
    docs_response = requests.get(docs_url)
    documents_raw = docs_response.json()

    for course in documents_raw:
        course_name = course['course']

        for doc in course['documents']:
            doc['course'] = course_name
            yield doc

In [ ]:
qdrant_destination = qdrant(
  qd_path="db.qdrant", 
)

In [14]:
pipeline = dlt.pipeline(
    pipeline_name="zoomcamp_pipeline",
    destination=qdrant_destination,
    dataset_name="zoomcamp_tagged_data"

)
load_info = pipeline.run(zoomcamp_data())
print(pipeline.last_trace)

Run started at 2025-07-07 03:59:45.069044+00:00 and COMPLETED in 9.25 seconds with 4 steps.
Step extract COMPLETED in 0.86 seconds.

Load package 1751860786.6822588 is EXTRACTED and NOT YET LOADED to the destination and contains no failed jobs

Step normalize COMPLETED in 0.11 seconds.
Normalized data for the following tables:
- _dlt_pipeline_state: 1 row(s)
- zoomcamp_data: 948 row(s)

Load package 1751860786.6822588 is NORMALIZED and NOT YET LOADED to the destination and contains no failed jobs

Step load COMPLETED in 6.68 seconds.
Pipeline zoomcamp_pipeline load step completed in 6.66 seconds
1 load package(s) were loaded to destination qdrant and into dataset zoomcamp_tagged_data
The qdrant destination used /home/radianv/projects/data_talk_club/local-llm-zoomcamp/workshops/dlt/code/local/db.qdrant location to store data
Load package 1751860786.6822588 is LOADED and contains no failed jobs

Step run COMPLETED in 9.25 seconds.
Pipeline zoomcamp_pipeline load step completed in 6.66 se